In [27]:
print('Hello LangChain')

Hello LangChain


In [28]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#print(OPENAI_API_KEY)

In [29]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("human", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="LangServe는 무엇인가요? 자세하게 설명해주세요")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [30]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x00000193A587B890> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000193A58797C0> root_client=<openai.OpenAI object at 0x00000193A588ED20> root_async_client=<openai.AsyncOpenAI object at 0x00000193A5878380> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [31]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print(response)
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
content='LangServe는 LangChain을 기반으로 구축된 오픈 소스 라이브러리입니다. LangChain은 자연어 처리(NLP) 및 인공지능(AI) 모델을 활용하여 애플리케이션을 구축하기 위한 프레임워크입니다. LangServe는 이 LangChain을 사용하여 API 서버를 쉽게 구축할 수 있도록 도와주는 라이브러리입니다.\n\nLangServe를 사용하면 LangChain으로 구축된 AI 모델을 RESTful API로 쉽게 변환할 수 있습니다. 이를 통해 개발자는 LangChain으로 구축된 모델을 웹 애플리케이션, 모바일 애플리케이션 또는 다른 시스템과 통합할 수 있습니다.\n\nLangServe의 주요 기능은 다음과 같습니다.\n\n*   **LangChain 통합**: LangServe는 LangChain과 긴밀하게 통합되어 있습니다. 따라서 LangChain으로 구축된 모델을 LangServe를 사용하여 API 서버로 쉽게 변환할 수 있습니다.\n*   **API 서버 구축**: LangServe는 API 서버를 구축하는 데 필요한 기능을 제공합니다. 이를 통해 개발자는 LangChain으로 구축된 모델을 RESTful API로 변환할 수 있습니다.\n*   **다양한 프로토콜 지원**: LangServe는 HTTP, gRPC 등 다양한 프로토콜을 지원합니다. 따라서 개발자는 다양한 클라이언트와 서버를 구축할 수 있습니다.\n\nLangServe를 사용하는 방법은 간단합니다. 먼저 LangChain으로 모델을 구축한 다음, LangServe를 사용하여 API 서버를 구축하면 됩니다. LangServe는 다양한 예제와 문서를 제공하므로, 개발자는 쉽게 LangServe를 시작할 수 있습니다.\n\n예를 들어, 다음과 같은 코드로 LangServe를 사용할 수 있습니다.\n\n```python\nfrom langserve import LangServe\n\n

### LCEL
* Prompt + LLM을 Chain으로 연결하기

In [32]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """
    You are an expert in AI Expert. Answer the question. 
    <Question>: {input}에 대해 쉽게 반드시 한글로 설명해주세요.")
    """)                                     
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    You are an expert in AI Expert. Answer the question. \n    <Question>: {input}에 대해 쉽게 반드시 한글로 설명해주세요.")\n    ')

In [33]:
# chain 연결 (LCEL) prompt + llm 연결
chain = prompt | llm
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


### LCEL
* Prompt + LLM + OutputParser을 Chain으로 연결하기

In [34]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL) prompt + llm + outputparser
output_parser = StrOutputParser()

chain2 = prompt | llm | output_parser
print(type(chain2))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [35]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리는 컴퓨터가 데이터를 통해 스스로 학습하고, 그 학습을 통해 특정 작업을 수행하는 능력을 갖추는 것입니다. 이를 위해 인공지능 모델은 크게 세 가지 단계, 즉 데이터 수집, 모델 훈련, 그리고 모델 평가 및 활용으로 구분할 수 있습니다.

### 1. 데이터 수집
인공지능 모델이 학습하기 위해서는 다양한 데이터가 필요합니다. 이러한 데이터는 이미지, 텍스트, 오디오 등 여러 형태가 될 수 있습니다. 예를 들어, 고양이와 개를 구분하는 모델을 만들기 위해서는 수많은 고양이와 개의 이미지 데이터가 필요합니다.

### 2. 모델 훈련
데이터 수집 후, 인공지능 모델을 훈련하는 과정이 필요합니다. 이 과정에서는 수집한 데이터를 바탕으로 모델이 스스로 특정 패턴이나 규칙을 발견하도록 합니다. 모델 훈련은 크게 지도 학습(Supervised Learning), 비지도 학습(Unsupervised Learning), 그리고 강화 학습(Reinforcement Learning)으로 나눌 수 있습니다.

- **지도 학습**: 모델이 이미 레이블이 지정된 데이터를 통해 학습하는 방법입니다. 예를 들어, 사진과 그에 따른 설명(예: "고양이", "개")을 통해 모델이 고양이와 개를 구별하는 법을 배웁니다.
- **비지도 학습**: 모델이 레이블이 지정되지 않은 데이터를 분석하여 스스로 패턴이나 관계를 찾는 방법입니다. 예를 들어, 비슷한 이미지끼리 그룹을 만드는 것입니다.
- **강화 학습**: 모델이 환경과 상호작용하며 보상을 최대화하는 방향으로 학습하는 방법입니다. 예를 들어, 주어진 게임에서 이기는 방법을 스스로 터득하는 것입니다.

### 3. 모델 평가 및 활용
모델 훈련이 완료되면, 실제 상황에서 얼마나 잘 작동하는지 평가합니다. 이를 위해 별도의 테스트 데이터 세트를 사용하여 모델의 성능을 측정합니다. 평가 결과에 따라 모델의 성능을 개선하기 위해 하이퍼파

In [36]:
# chain 호출
try:
    result = chain2.invoke({"input": ": LangChain의 Products(제품)는 어떤 것들이 있나요? 예를 들어 LangSmith, LangServe 같은 Product가 있어"})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 인공지능(AI) 모델을 활용하여 자연어 처리(NLP) 및 대화형 AI 애플리케이션을 개발할 수 있는 플랫폼입니다. LangChain의 주요 제품은 다음과 같습니다.

1. **LangSmith**: 랭스미스는 랭체인에서 제공하는 대화형 AI 모델 개발 및 관리 플랫폼입니다. 이 플랫폼을 통해 개발자는 대화형 AI 모델을 쉽게 개발, 테스트, 배포할 수 있습니다. 랭스미스는 모델의 성능을 모니터링하고, 데이터 세트를 관리하며, 모델을 개선할 수 있는 다양한 도구를 제공합니다.

2. **LangServe**: 랭서브는 랭체인에서 제공하는 API 서비스입니다. 이 서비스를 통해 개발자는 랭체인에서 개발한 대화형 AI 모델을 쉽게 API로 배포하고, 다른 애플리케이션에 통합할 수 있습니다. 랭서브는 모델의 성능을 최적화하고, 대규모 트래픽을 처리할 수 있는 인프라를 제공합니다.

3. **LangChain 자체**: 랭체인은 랭스미스, 랭서브 등 여러 제품과 서비스를 포함하는 통합 플랫폼입니다. 랭체인은 개발자가 대화형 AI 애플리케이션을 개발할 수 있는 다양한 도구와 라이브러리를 제공합니다. 개발자는 랭체인을 통해 다양한 AI 모델을 활용하여 자연어 처리, 대화 관리, 응답 생성 등 다양한 기능을 구현할 수 있습니다.

이러한 제품과 서비스를 통해 LangChain은 개발자가 대화형 AI 애플리케이션을 쉽게 개발하고 배포할 수 있도록 지원합니다.


### Runnable의 stream() 함수 호출

In [37]:
# 스트리밍 출력을 위한 요청
try:
    answer = chain2.stream({"input": "인공지능 모델의 학습 원리를 자세하게 설명해 주세요."})
    
    # 스트리밍 출력
    #print(answer)
    for token in answer:
        # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
        print(token, end="", flush=True)
except Exception as e:
    print(f"오류 발생: {e}")


인공지능 모델의 학습 원리를 설명드리겠습니다.

인공지능 모델의 학습은 크게 세 가지 단계로 이루어져 있습니다: 데이터 수집, 모델 훈련, 모델 평가입니다.

**1. 데이터 수집**

인공지능 모델을 학습시키기 위해서는 먼저 데이터를 수집해야 합니다. 이 데이터는 모델이 학습해야 할 패턴이나 관계를 포함하고 있어야 합니다. 예를 들어, 이미지 분류 모델을 만든다면, 다양한 이미지들을 수집해야 합니다.

**2. 모델 훈련**

수집된 데이터를 바탕으로 모델을 훈련시킵니다. 모델 훈련은 모델이 데이터를 분석하고 학습하여 최적의 파라미터를 찾는 과정입니다. 이 과정은 다음과 같은 단계로 이루어집니다:

*   **데이터 전처리**: 수집된 데이터를 모델에 입력하기 전에 처리하는 단계입니다. 이 단계에서는 데이터의 노이즈를 제거하거나, 데이터를 정규화하는 등의 작업이 이루어집니다.
*   **모델 초기화**: 모델의 파라미터를 초기화하는 단계입니다. 이 단계에서는 모델의 가중치와 편향 등을 초기화합니다.
*   **순전파**: 모델에 입력 데이터를 넣고 출력값을 계산하는 단계입니다. 이 단계에서는 모델의 가중치와 편향 등을 사용하여 출력값을 계산합니다.
*   **역전파**: 출력값과 실제값 사이의 오류를 계산하고, 모델의 파라미터를 업데이트하는 단계입니다. 이 단계에서는 오류를 최소화하기 위해 모델의 가중치와 편향 등을 업데이트합니다.

**3. 모델 평가**

모델 훈련이 완료되면, 모델을 평가합니다. 모델 평가에서는 모델의 성능을 측정하고, 모델이 얼마나 잘 학습했는지 확인합니다. 이 단계에서는 평가 데이터셋을 사용하여 모델의 성능을 측정합니다.

예를 들어, 이미지 분류 모델의 경우, 평가 데이터셋에 대한 정확도를 측정할 수 있습니다. 또한, 모델의 성능을 향상시키기 위해 하이퍼파라미터를 조정하거나, 모델의 구조를 변경할 수 있습니다.

결론적으로, 인공지능 모델의 학습은 데이터 수집, 모델 훈련, 모델 평가의 세 가지 단계로 이루어져 있습니다. 모델 훈련

### Multi Chain
* 첫번째 Chain의 출력이, 두번째 Chain의 입력이 된다.
* 두개의 Chain과 Prompt + OutputParser를 LCEL로 연결하기

In [38]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 한국 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 10문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

In [39]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 입력 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )

    # 실행: "SF" 장르의 영화 추천 및 줄거리 요약
    response = chain2.invoke({"genre": "액션"})
    print(response)  
except Exception as e:
    print(f"오류 발생: {e}")

영화의 제목은 '추격자'입니다.

부산에서 서울로 귀향하는 택시기사 김윤석과 그를 쫓는 강력계 형사 하정우의 이야기를 다루는 영화입니다.
이 영화는 두 남자의 긴박한 추격과 추격전을 중심으로 전개됩니다.
김윤석은 택시기사로, 하정우는 강력계 형사로 등장합니다.
하정우는 김윤석을 쫓으며 긴장감 넘치는 액션과 추격 장면을 선보입니다.
김윤석과 하정우의 연기는 많은 이들의 찬사를 받았습니다.
영화는 한국 영화계의 대표적인 액션 영화 중 하나로 꼽힙니다.
강렬한 액션과 긴장감 넘치는 스토리가 어우러져 많은 관객들에게 추천할 만한 영화입니다.
'추격자'는 2008년에 개봉하여 큰 호평을 받았습니다.
이 영화는 두 남자의 대결을 통해 긴장감과 액션을 선사합니다.


### PromptTemplate 여러개 연결하기

In [40]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 요약해서 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="영어")

# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"영어"})
print(response)

ChatGPT 모델의 학습 원리는 다음과 같습니다.

ChatGPT는 대규모 언어 데이터셋을 기반으로 훈련된 생성형 언어 모델입니다. 이 모델은 주어진 문맥에 따라 다음에 올 가능성이 높은 단어를 예측하도록 학습하며, 이를 통해 자연스러운 텍스트를 생성할 수 있습니다. 학습 과정에서는 많은 양의 텍스트 데이터를 분석하고 패턴을 학습하여 언어에 대한 이해를 높입니다.

ChatGPT 모델의 장점은 다음과 같습니다.

* 자연스러운 텍스트 생성: ChatGPT는 사람과 유사한 자연스러운 텍스트를 생성할 수 있습니다.
* 다양한 주제 이해: 대규모 언어 데이터셋을 기반으로 훈련되어 다양한 주제에 대한 이해도가 높습니다.
* 대화형 인터페이스: 사용자와 대화형식으로 상호작용할 수 있어 질문에 답변하거나 정보를 제공할 수 있습니다.

ChatGPT 모델과 비슷한 AI 모델은 다음과 있습니다.

* LLaMA
* PaLM
* BERT
* RoBERTa


In [43]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 2},
    {"model_name": "Gemma", "count": 3},
    {"model_name": "llma-4","count": 4},
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions]
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

['GPT-4 모델의 학습 원리를 2 문장으로 한국어로 답변해 주세요.', 'Gemma 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'llma-4 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']


In [44]:
# OpenAI 모델 사용
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

for prompt in formatted_prompts:
    response = llm.invoke(prompt)
    print(response)

content='GPT-4 모델은 대규모의 텍스트 데이터를 학습하여 언어 패턴과 구조를 이해하고, 이를 기반으로 자연어 처리 작업을 수행합니다. GPT-4는 이전 모델인 GPT-3와 유사한 방식으로 트랜스포머 아키텍처를 사용하여 학습되었으며, 더 많은 양의 데이터와 컴퓨팅 리소스를 활용하여 성능을 개선했습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 28, 'total_tokens': 105, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.36129548300000003, 'prompt_time': 0.002948421, 'completion_time': 0.183964473, 'total_time': 0.186912894}, 'model_name': 'meta-llama/llama-4-scout-17b-16e-instruct', 'system_fingerprint': 'fp_37da608fc1', 'id': 'chatcmpl-2ec9edc1-117f-4371-b98a-dc76a5cc4539', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--f87aec03-1be7-4183-adb4-14c946468158-0' usage_metadata={'input_tokens': 28, 'output_tokens': 77, 'total_tokens': 105, 'input_token_details': {}, 'output_token_details': {}}
content='Gemma는 컴퓨터가 자연어 텍스트를 생성, 이해 및 조작하는 방법을 배우는 언어 모델입니다. 텍스트의 통계적 패턴을 분석하여 학습한 다음 이 지식을 사용하여